In [1]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
from datetime import datetime
pd.options.display.max_columns = None
from pybaseball import batting_stats, pitching_stats, cache, playerid_lookup, statcast_batter, statcast_pitcher, statcast, batting_stats_range
import math
cache.enable()
cache.config.cache_type='csv'
cache.config.save()
from fantasy_utils import Fantasy_Projections as fp
import fantasy_utils as fu
fp = fu.Fantasy_Projections()
import optimize_lineup as ol

In [86]:
eligibility_url_dict = {"9 Grand Kids":'30', 'Brewbirds':'3', 'Charmer':'36', 'Dirty Birds':'41', "Harveys Wallbangers":'4', 'Lil Trump & the Ivanabees':'27', 'Lima Time!':'38', 
                        'Roid Ragers':'44', 'Trouble with the Curve':'1', 'Ugly Spuds':'29', 'Wiscompton Wu-Tang':'42', 'Young Guns':'45'}
name_change = {"9 Gran...":'9 Grand Kids', 'Brewbi...':'Brewbirds', 'Charme...':'Charmer', 'Dirty ...':'Dirty Birds', 'Harvey...':'Harvey', 'Lil Tr...':'Lil Trump', 
               'Lima T...':'Lima Time', 'Roid R...':'Roiders', 'Troubl...':'Trouble', 'Ugly S...':'Ugly Spuds', 'Wiscom...':'Wu Tang', 'Young ...':'Young Guns'}
name_change2 = {'Harvey':'Harveys Wallbangers', 'Lil Trump':'Lil Trump & the Ivanabees', 'Lima Time':'Lima Time!', 'Roiders':'Roid Ragers', 'Trouble':'Trouble with the Curve',
                'Wu Tang':'Wiscompton Wu-Tang', 'Madness':'9 Grand Kids'}
position_priority = ['C', '2B', '3B', 'SS', 'OF', '1B', 'MI', 'CI', 'DH', 'SP', 'RP']

def stitch_positions(row):
    pos_code = row[position_priority+['P']]>=5
    return list(pos_code[pos_code].index)


In [13]:
ids = fp.load_id_map()

In [6]:
df = pd.read_excel('data//2023-owner-weekly-sit-start.xlsx')
df = df[df['Week']==1]

In [209]:
h = pd.read_excel('data//2023-weekly-stats-h.xlsx')

In [210]:
h.CBSNAME.fillna(h['Player'],inplace=True)

In [170]:
h = h[h['Owner']!='FA']
h.drop(columns=['all_pos', 'Avail'],inplace=True)

In [171]:
df = pd.DataFrame()

for wk in range(1,28):
    if wk<=14:
        proj = pd.read_csv(f'data//period_14_projections.csv')
    elif wk==24:
        proj = pd.read_csv(f'data//period_23_projections.csv')
    else:
        proj = pd.read_csv(f'data//period_{wk}_projections.csv')
    proj.Avail.replace(name_change,inplace=True)
    proj.Avail.replace(name_change2,inplace=True)
    proj.Owner.replace(name_change2,inplace=True)
    proj['Week'] = wk
    df = pd.concat([df, h.merge(proj[['Owner', 'Week', 'Player', 'all_pos']], on=['Owner', 'Week', 'Player'], how='inner')])
    
#df.rename(columns={'all_pos_y':'all_pos'},inplace=True)
#df.drop(columns='all_pos_x', inplace=True)

In [172]:
h.merge(proj[['Owner', 'Week', 'Player', 'all_pos']], on=['Owner', 'Week', 'Player'], how='inner').query('Player=="Jorge Mateo"')

,Player,AB,R,H,1B,2B,3B,HR,RBI,BB,K,SB,CS,AVG,OBP,SLG,Rank,Week,Team,Owner,Pos,Decision,CBSNAME,CBSID,IDFANGRAPHS,MLBID,zBA_cnt,zR,zRBI,zHR,zSB,zBA,z,all_pos
42,Jorge Mateo,9.0,3.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,3.0,0.0,0.222,0.3,0.222,306.0,27,BAL,Roid Ragers,DH,Sit,Jorge Mateo,2167484.0,17273,622761.0,-0.178129,0.72006,-0.612225,-0.477692,5.511973,-0.111949,5.030167,"['SS', 'MI', 'DH']"


In [173]:
x = h.merge(df[['Player', 'Owner', 'Week', 'all_pos']], on=['Player', 'Owner', 'Week'], how='outer', indicator=True)

In [174]:
x._merge.value_counts()

both          5580
left_only      525
right_only       0
Name: _merge, dtype: int64

In [196]:
p = 'Jorge Mateo'
x.loc[x['Player']==p, 'all_pos'] = x.loc[x['Player']==p, 'all_pos'].bfill()

In [197]:
x.loc[x['Player']==p, 'all_pos']#.bfill()

10      ['SS', 'MI', 'DH']
246     ['SS', 'MI', 'DH']
462     ['SS', 'MI', 'DH']
833     ['SS', 'MI', 'DH']
910     ['SS', 'MI', 'DH']
1232    ['SS', 'MI', 'DH']
1584    ['SS', 'MI', 'DH']
1693    ['SS', 'MI', 'DH']
1966    ['SS', 'MI', 'DH']
2133    ['SS', 'MI', 'DH']
2369    ['SS', 'MI', 'DH']
2586    ['SS', 'MI', 'DH']
2858    ['SS', 'MI', 'DH']
3105    ['SS', 'MI', 'DH']
3383    ['SS', 'MI', 'DH']
3552    ['SS', 'MI', 'DH']
3762    ['SS', 'MI', 'DH']
4052    ['SS', 'MI', 'DH']
4164    ['SS', 'MI', 'DH']
4460    ['SS', 'MI', 'DH']
4639    ['SS', 'MI', 'DH']
4897    ['SS', 'MI', 'DH']
5136    ['SS', 'MI', 'DH']
5285    ['SS', 'MI', 'DH']
5552    ['SS', 'MI', 'DH']
5798    ['SS', 'MI', 'DH']
5921    ['SS', 'MI', 'DH']
Name: all_pos, dtype: object

In [198]:
for p in x[x['_merge']=='left_only']['Player'].unique():
    x.loc[x['Player']==p, 'all_pos'] = x.loc[x['Player']==p, 'all_pos'].bfill()

In [201]:
x[x['all_pos'].isna()].sort_values('Player')

,Player,AB,R,H,1B,2B,3B,HR,RBI,BB,K,SB,CS,AVG,OBP,SLG,Rank,Week,Team,Owner,Pos,Decision,CBSNAME,CBSID,IDFANGRAPHS,MLBID,zBA_cnt,zR,zRBI,zHR,zSB,zBA,z,all_pos,_merge
2005,Adalberto Mondesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,BOS,Brewbirds,DH,Sit,Adalberto Mondesi,2044531.0,13769,609275.0,0.000000,-1.099651,-0.687028,-0.477692,-0.346043,-5.544850e-16,-2.610414,NaN,left_only
166,Adalberto Mondesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,BOS,Brewbirds,DH,Sit,Adalberto Mondesi,2044531.0,13769,609275.0,0.000000,-0.876334,-0.477692,-0.477692,-0.346043,6.419298e-16,-2.177762,NaN,left_only
1324,Adalberto Mondesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,BOS,Brewbirds,DH,Sit,Adalberto Mondesi,2044531.0,13769,609275.0,0.000000,-1.147614,-0.676834,-0.477692,-0.346043,5.427357e-16,-2.648183,NaN,left_only
1774,Adalberto Mondesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,BOS,Brewbirds,DH,Sit,Adalberto Mondesi,2044531.0,13769,609275.0,0.000000,-1.132907,-0.630322,-0.477692,-0.346043,1.744637e-16,-2.586964,NaN,left_only
2223,Adalberto Mondesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,BOS,Brewbirds,DH,Sit,Adalberto Mondesi,2044531.0,13769,609275.0,0.000000,-1.188880,-0.608235,-0.477692,-0.346043,-9.346223e-16,-2.620851,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,Wil Myers,27.0,5.0,9.0,6.0,1.0,0.0,2.0,8.0,1.0,7.0,1.0,0.0,0.333,0.357,0.593,40.0,3,CIN,Wiscompton Wu-Tang,OF,Start,Wil Myers,1744704.0,10047,571976.0,2.325419,1.547164,1.781970,3.258712,1.606629,1.367359e+00,9.561835,NaN,left_only
1974,Wil Myers,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.000,0.250,0.000,644.0,9,CIN,Wiscompton Wu-Tang,OF,Sit,Wil Myers,1744704.0,10047,571976.0,-1.493414,-1.099651,-0.687028,-0.477692,1.606629,-8.087685e-01,-1.466511,NaN,left_only
1251,Wil Myers,8.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0,0.125,0.125,0.500,590.0,6,CIN,Wiscompton Wu-Tang,CI,Sit,Wil Myers,1744704.0,10047,571976.0,-1.014768,-0.617248,0.564534,1.390510,-0.346043,-6.116729e-01,0.380080,NaN,left_only
155,Zac Veen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,COL,Harveys Wallbangers,DH,Sit,Zac Veen,3173109.0,sa3014516,691728.0,0.000000,-0.876334,-0.477692,-0.477692,-0.346043,6.419298e-16,-2.177762,NaN,left_only


In [214]:
x

,Unnamed: 0,Player,AB,R,H,1B,2B,3B,HR,RBI,BB,K,SB,CS,AVG,OBP,SLG,Rank,Week,Team,Owner,Pos,Decision,CBSNAME,CBSID,IDFANGRAPHS,MLBID,zBA_cnt,zR,zRBI,zHR,zSB,zBA,z,all_pos,_merge
0,4929,Josh Naylor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,CLE,Lima Time!,1B,Sit,Josh Naylor,2184448,18839,647304,0.000000,-1.126029,-0.678882,-0.477692,-0.346043,2.590000e-17,-2.628647,"['1B', 'CI', 'DH', 'P']",both
1,3818,Rowdy Tellez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,MIL,Lil Trump & the Ivanabees,CI,Start,Rowdy Tellez,2171831,15679,642133,0.000000,-1.133759,-0.707535,-0.477692,-0.346043,-5.150000e-16,-2.665028,"['1B', 'CI', 'DH', 'P']",both
2,4041,Rowdy Tellez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,MIL,Lil Trump & the Ivanabees,CI,Start,Rowdy Tellez,2171831,15679,642133,0.000000,-1.149724,-0.630260,-0.477692,-0.346043,3.740000e-16,-2.603720,"['1B', 'CI', 'DH', 'P']",both
3,112,Andrew Vaughn,11.0,1.0,3.0,1.0,2.0,0.0,0.0,3.0,1.0,3.0,0.0,0.0,0.273,0.429,0.455,410.0,1,CHW,Young Guns,OF,Start,Andrew Vaughn,3117474,26197,683734,0.299639,-0.054120,-0.477692,-0.477692,-0.346043,2.289851e-01,-1.126563,"['1B', 'CI', 'DH']",both
4,359,Andrew Vaughn,22.0,4.0,6.0,4.0,2.0,0.0,0.0,4.0,5.0,5.0,0.0,0.0,0.273,0.407,0.364,390.0,2,CHW,Young Guns,OF,Start,Andrew Vaughn,3117474,26197,683734,0.469848,0.935050,-0.663267,-0.477692,-0.346043,2.831805e-01,-0.268772,"['1B', 'CI', 'DH']",both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6100,2220,Wil Myers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,CIN,Wiscompton Wu-Tang,OF,Sit,Wil Myers,1744704,10047,571976,0.000000,-1.188880,-0.608235,-0.477692,-0.346043,-9.350000e-16,-2.620851,"['OF','1B','CI','DH']",left_only
6101,2453,Wil Myers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,CIN,Wiscompton Wu-Tang,OF,Sit,Wil Myers,1744704,10047,571976,0.000000,-1.143594,-0.670565,-0.477692,-0.346043,3.730000e-16,-2.637894,"['OF','1B','CI','DH']",left_only
6102,2677,Wil Myers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,CIN,Wiscompton Wu-Tang,OF,Sit,Wil Myers,1744704,10047,571976,0.000000,-1.095248,-0.631028,-0.477692,-0.346043,-2.620000e-16,-2.550011,"['OF','1B','CI','DH']",left_only
6103,155,Zac Veen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,COL,Harveys Wallbangers,DH,Sit,Zac Veen,3173109,sa3014516,691728,0.000000,-0.876334,-0.477692,-0.477692,-0.346043,6.420000e-16,-2.177762,"['OF','DH']",left_only


In [204]:
x = pd.read_csv('test.csv')

In [207]:
x.to_excel('data//2023-weekly-stats-h-new.xlsx')

In [212]:
h.drop(columns='all_pos',inplace=True)

In [216]:
h = h.merge(x[['Player', 'Owner', 'Week', 'all_pos']], on=['Player', 'Owner', 'Week'], how='left')

In [220]:
h.to_excel('data//2023-weekly-stats-h-new.xlsx')